# Convert mw NetCDF to Tif

**Purpose of script:**

Convert the microwave data from NetCDF format to Tif format. 

- In: mw NetCDF data (one file)
- Out: mw data in Tif foramt (multiple files)

*Note: Dates are specified by defining the start and end date of desired period, the resulting date list of dates in defined range is passed to the function as an argument*

In [1]:
import numpy as np
import xarray
import rasterio
import pandas as pd

## Define conversion functions

In [2]:
def datelist(start_date, end_date):
    datelist = pd.date_range(start_date, end_date)
    return datelist

In [3]:
def convert_to_tif(path_in, path_out, datelist):
    # Load data:
    dataset_mw = xarray.open_dataset(path_in).rio.write_crs("epsg:3413", inplace=True)

    for i in datelist:
        print(i)
        dataset_day = dataset_mw.where(dataset_mw["time"] == i, drop=True)
        data_mw_day = dataset_day["melt"]
        out_path_and_name = f"{path_out}{i.date()}_mw.tif"
        data_mw_day.rio.to_raster(out_path_and_name)

        with rasterio.open(out_path_and_name) as src_mw:
            mw = src_mw.read()
            np.nan_to_num(mw, nan=-1, copy= False) # inplace
            kwargs1 = src_mw.meta.copy()

        # overwrite mw with new band (fixed nan)
        with rasterio.open(out_path_and_name, 'w', **kwargs1) as dst:
            dst.write_band(1, mw[0])
    return 
# would be nice to do that with just one package

## Run conversion

In [6]:
start_dates = ["2017-05-01", "2018-05-01", "2019-05-01"]
end_dates = ["2017-10-30", "2018-10-30", "2019-10-30"]  # note that 31st October is not included (broken??)

mw_file = r"../Data/microwave-rs/CumJour-North-ssmi-1979-2022-H19.nc"
mw_out_path = r'../Data/microwave-rs/mw_tif/'

In [7]:
for start_date, end_date in zip(start_dates, end_dates):
    convert_to_tif(mw_file, mw_out_path, datelist(start_date, end_date))

2017-05-01 00:00:00
2017-05-02 00:00:00
2017-05-03 00:00:00
2017-05-04 00:00:00
2017-05-05 00:00:00
2017-05-06 00:00:00
2017-05-07 00:00:00
2017-05-08 00:00:00
2017-05-09 00:00:00
2017-05-10 00:00:00
2017-05-11 00:00:00
2017-05-12 00:00:00
2017-05-13 00:00:00
2017-05-14 00:00:00
2017-05-15 00:00:00
2017-05-16 00:00:00
2017-05-17 00:00:00
2017-05-18 00:00:00
2017-05-19 00:00:00
2017-05-20 00:00:00
2017-05-21 00:00:00
2017-05-22 00:00:00
2017-05-23 00:00:00
2017-05-24 00:00:00
2017-05-25 00:00:00
2017-05-26 00:00:00
2017-05-27 00:00:00
2017-05-28 00:00:00
2017-05-29 00:00:00
2017-05-30 00:00:00
2017-05-31 00:00:00
2017-06-01 00:00:00
2017-06-02 00:00:00
2017-06-03 00:00:00
2017-06-04 00:00:00
2017-06-05 00:00:00
2017-06-06 00:00:00
2017-06-07 00:00:00
2017-06-08 00:00:00
2017-06-09 00:00:00
2017-06-10 00:00:00
2017-06-11 00:00:00
2017-06-12 00:00:00
2017-06-13 00:00:00
2017-06-14 00:00:00
2017-06-15 00:00:00
2017-06-16 00:00:00
2017-06-17 00:00:00
2017-06-18 00:00:00
2017-06-19 00:00:00


## Testing Lina

In [2]:
mw_file = r"../Data/microwave-rs/CumJour-North-ssmi-1979-2022-H19.nc"
mw_out_path_test = r'../Data/microwave-rs/mw_tif_non_melt_testing/'

In [3]:
period = [['2017-06-01', '2017-08-31'],
          ['2018-06-01', '2018-08-31'],
          ['2019-06-01', '2019-08-31'], 
          ['2020-06-01', '2020-08-31'],
          ['2021-06-01', '2021-08-31'],
          ['2022-06-01', '2022-08-31']]

datelist_test = []
for i in period:
    datelist_ = list(pd.date_range(*i))
    datelist = datelist + datelist_

In [5]:
convert_to_tif(mw_file, mw_out_path_test, datelist(start_date, datelist_test))

2017-06-01 00:00:00
2017-06-02 00:00:00
2017-06-03 00:00:00
2017-06-04 00:00:00
2017-06-05 00:00:00
2017-06-06 00:00:00
2017-06-07 00:00:00
2017-06-08 00:00:00
2017-06-09 00:00:00
2017-06-10 00:00:00
2017-06-11 00:00:00
2017-06-12 00:00:00
2017-06-13 00:00:00
2017-06-14 00:00:00
2017-06-15 00:00:00
2017-06-16 00:00:00
2017-06-17 00:00:00
2017-06-18 00:00:00
2017-06-19 00:00:00
2017-06-20 00:00:00
2017-06-21 00:00:00
2017-06-22 00:00:00
2017-06-23 00:00:00
2017-06-24 00:00:00
2017-06-25 00:00:00
2017-06-26 00:00:00
2017-06-27 00:00:00
2017-06-28 00:00:00
2017-06-29 00:00:00
2017-06-30 00:00:00
2017-07-01 00:00:00
2017-07-02 00:00:00
2017-07-03 00:00:00
2017-07-04 00:00:00
2017-07-05 00:00:00
2017-07-06 00:00:00
2017-07-07 00:00:00
2017-07-08 00:00:00
2017-07-09 00:00:00
2017-07-10 00:00:00
2017-07-11 00:00:00
2017-07-12 00:00:00
2017-07-13 00:00:00
2017-07-14 00:00:00
2017-07-15 00:00:00
2017-07-16 00:00:00
2017-07-17 00:00:00
2017-07-18 00:00:00
2017-07-19 00:00:00
2017-07-20 00:00:00
